In [38]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import plotly.graph_objects as go
import plotly.express as px
import plotly.offline as py
py.init_notebook_mode(connected=True)
import warnings
warnings.filterwarnings("ignore")
from scipy.stats import ttest_ind, ttest_1samp
import gc

SMOOTHING = 3

In [39]:
def clean_data(df):
    if "Unnamed: 0" in df.columns:
        df.drop("Unnamed: 0", axis=1, inplace=True)
        
    for col in df.columns:
        df[col] = df[col].astype(str).str.rstrip('%').str.replace(',', '')
        try:
            df[col] = df[col].astype(float)
        except:
            pass
        
    df.set_index(df.columns[0], inplace=True)
    
    return df

example_map_df = clean_data(pd.read_csv("https://raw.githubusercontent.com/meryembarkallah21/valorant/master/map_data/maps_competitive_tier%3D3.csv"))
example_map_df

,Play Rate,Atk Win,Def Win,Num Matches
Map Name,,,,
Icebox,16.5,51.6,48.4,4743.0
Haven,17.2,51.2,48.8,4947.0
Breeze,15.9,51.2,48.8,4590.0
Bind,18.4,50.9,49.1,5304.0
Ascent,13.6,48.2,51.8,3927.0
Split,18.4,47.1,52.9,5304.0


In [40]:
general_ranks_list = ["Iron", "Bronze", "Silver", "Gold", "Platinum", "Diamond"]
ranks_list = np.array([f"{rank} {num}" for rank in general_ranks_list for num in range(1, 4)])
num_ranks = len(ranks_list)
print(f"Ranks: {ranks_list}, length: {num_ranks}")

Ranks: ['Iron 1' 'Iron 2' 'Iron 3' 'Bronze 1' 'Bronze 2' 'Bronze 3' 'Silver 1'
 'Silver 2' 'Silver 3' 'Gold 1' 'Gold 2' 'Gold 3' 'Platinum 1'
 'Platinum 2' 'Platinum 3' 'Diamond 1' 'Diamond 2' 'Diamond 3'], length: 18


Maps
Play Rate
Attack Win Rate

In [41]:
playrate_map_list = []
atkwin_map_list = []
matches_list = []
for i in range(3, 3 + num_ranks):
    tmp_map_df = clean_data(pd.read_csv(f"https://raw.githubusercontent.com/meryembarkallah21/valorant/master/map_data/maps_competitive_tier%3D{i}.csv"))    
    playrate_map_list.append(tmp_map_df['Play Rate'])
    atkwin_map_list.append(tmp_map_df['Atk Win'])
    matches_list.append(tmp_map_df['Num Matches'].sum())

playrate_df = pd.DataFrame(playrate_map_list, index=ranks_list)
atkwin_df = pd.DataFrame(atkwin_map_list, index=ranks_list)

custom_map_df = clean_data(pd.read_csv("https://raw.githubusercontent.com/meryembarkallah21/valorant/master/map_data/maps_custom_tier%3D0.csv"))
unrated_map_df = clean_data(pd.read_csv("https://raw.githubusercontent.com/meryembarkallah21/valorant/master/map_data/maps_unrated_tier%3D0.csv"))
spikerush_map_df = clean_data(pd.read_csv("https://raw.githubusercontent.com/meryembarkallah21/valorant/master/map_data/maps_spikerush_tier%3D0.csv"))

Play Rate

In [42]:
#general stats
avg_playrate_df = playrate_df.mean()
avg_playrate_df

Map Name
Icebox    16.638889
Haven     17.300000
Breeze    13.477778
Bind      17.494444
Ascent    17.861111
Split     17.194444
dtype: float64

In [43]:
def quick_stats(df, name):
    std = np.std(df.values)
    confidence = (1.96*std)
    mean = df.mean().round(3)
    low = (mean-confidence).round(3)
    high = (mean+confidence).round(3)
    print(f"95% Confidence Interval for {name}: {mean} +- {confidence.round(3)}; ({low}, {high})")

Without Smoothing
Below is a line plot of the exact playrates for each map given a player's rank.

In [44]:
fig = px.line(playrate_df)
fig.update_layout(
    title={'text': f"Map Play Rate Without Smoothing", 'x': 0.5,
                         'xanchor': 'center', 'font': {'size': 16}},
    xaxis_title="", yaxis_title="Play Rate"
)
fig.show()

With Smoothing
Smoothing is defined in the following example:

In [45]:
def smoother(df, smoothing=SMOOTHING):
    indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=smoothing)
    smoothed_df = (df.rolling(window=indexer).sum() + df.rolling(window=smoothing).sum() - df)/(2*smoothing-1)
    smoothed_df.fillna(df, inplace=True)
    return smoothed_df

smoothed_playrate_df = smoother(playrate_df)
smoothed_playrate_df

Map Name,Icebox,Haven,Breeze,Bind,Ascent,Split
Iron 1,16.50,17.20,15.90,18.40,13.60,18.40
Iron 2,15.50,19.40,13.30,17.40,15.50,19.00
Iron 3,16.58,17.52,14.14,17.48,15.74,18.56
Bronze 1,17.00,17.46,13.48,16.94,16.54,18.58
Bronze 2,17.22,16.98,13.60,16.88,16.98,18.32
Bronze 3,17.18,17.12,13.52,16.54,17.56,18.02
Silver 1,17.04,17.22,13.42,17.04,17.72,17.50
Silver 2,16.94,17.00,13.22,17.32,17.88,17.54
Silver 3,16.54,17.20,13.28,17.96,17.80,17.12
Gold 1,16.48,17.20,13.28,18.10,17.88,16.98


In [46]:
fig = px.line(smoothed_playrate_df)
fig.update_layout(
    title={'text': f"Map Play Rate With Smoothing = {SMOOTHING}", 'x': 0.5,
                         'xanchor': 'center', 'font': {'size': 16}},
    xaxis_title="", yaxis_title="Play Rate"
)
fig.show()

Non-Competitive Play
Includes modes such as Customs, Unrated, and Spike Rush.

In [47]:
def create_other_df(index):
    other_df = pd.DataFrame(custom_map_df[index].sort_values(ascending=False))
    other_df.columns = ["Custom"]
    other_df['Unrated'] = unrated_map_df[index]
    other_df['Spike Rush'] = spikerush_map_df[index]
    return other_df
    
other_playrate_df = create_other_df('Play Rate')

In [48]:
fig = px.bar(other_playrate_df, barmode="group", labels={"variable": "Mode"})
fig.update_layout(
    title={'text': f"Map Play Rate for Other Modes", 'x': 0.5,
                         'xanchor': 'center', 'font': {'size': 16}},
    xaxis_title="", yaxis_title="Play Rate"
)
fig.show()

In [49]:
quick_stats(avg_playrate_df, "Competitive")
quick_stats(unrated_map_df['Play Rate'], "Unrated")
quick_stats(spikerush_map_df['Play Rate'], "Spike Rush")
quick_stats(custom_map_df['Play Rate'], "Custom")

95% Confidence Interval for Competitive: 16.661 +- 2.881; (13.78, 19.542)
95% Confidence Interval for Unrated: 16.683 +- 0.486; (16.197, 17.169)
95% Confidence Interval for Spike Rush: 16.65 +- 0.852; (15.798, 17.502)
95% Confidence Interval for Custom: 16.667 +- 21.769; (-5.102, 38.436)


Attack Win Rate
Certain maps may be T or CT-sided.

In [50]:
avg_atkwin_df = atkwin_df.mean()
avg_atkwin_df

Map Name
Icebox    49.455556
Haven     51.050000
Breeze    50.616667
Bind      48.961111
Ascent    46.688889
Split     47.577778
dtype: float64

In [51]:
fig = px.line(atkwin_df)
fig.add_shape(type='line',
                x0=0,
                y0=50,
                x1=num_ranks-1,
                y1=50,
                line=dict(color='Black', dash='dashdot'),
                xref='x',
                yref='y'
)
fig.update_layout(
    title={'text': f"Map Attack Win Rate Without Smoothing", 'x': 0.5,
                         'xanchor': 'center', 'font': {'size': 16}},
    xaxis_title="", yaxis_title="Attack Win Rate"
)
fig.show()

In [52]:
smoothed_atkwin_df = smoother(atkwin_df)
fig = px.line(smoothed_atkwin_df)
fig.add_shape(type='line',
                x0=0,
                y0=50,
                x1=num_ranks-1,
                y1=50,
                line=dict(color='Black', dash='dashdot'),
                xref='x',
                yref='y'
)
fig.update_layout(
    title={'text': f"Attack Win Rate With Smoothing", 'x': 0.5,
                         'xanchor': 'center', 'font': {'size': 16}},
    xaxis_title="", yaxis_title="Attack Win Rate"
)
fig.show()

In [53]:
other_atkwin_df = create_other_df('Atk Win')
fig = px.bar(other_atkwin_df, barmode="group", labels={"variable": "Mode"})
fig.update_layout(
    title={'text': f"Map Play Rate for Other Modes", 'x': 0.5,
                         'xanchor': 'center', 'font': {'size': 16}},
    yaxis_title="Play Rate"
)
fig.show()

In [54]:
quick_stats(avg_atkwin_df, "Competitive")
quick_stats(unrated_map_df['Atk Win'], "Unrated")
quick_stats(spikerush_map_df['Atk Win'], "Spike Rush")
quick_stats(custom_map_df['Atk Win'], "Custom")

95% Confidence Interval for Competitive: 49.058 +- 3.033; (46.025, 52.091)
95% Confidence Interval for Unrated: 50.583 +- 1.925; (48.658, 52.508)
95% Confidence Interval for Spike Rush: 51.183 +- 1.84; (49.343, 53.023)
95% Confidence Interval for Custom: 49.483 +- 4.454; (45.029, 53.937)


Number of Matches for Each Rank
Which rank has the most matches / people playing?

In [55]:
matches_df = pd.DataFrame(matches_list, index=ranks_list, columns=["Matches"])
fig = px.line(matches_df)
fig.update_layout(
    title={'text': f"Number of Matches", 'x': 0.5,
                         'xanchor': 'center', 'font': {'size': 16}},
    xaxis_title="", yaxis_title="Matches", showlegend=False
)
fig.show()

In [56]:
matches_df['cum_matches'] = matches_df.cumsum()
total_matches = matches_df['Matches'].sum()
fig = px.line(matches_df['cum_matches']/total_matches)
fig.add_shape(type='line',
                x0=0,
                y0=0.5,
                x1=num_ranks-1,
                y1=0.5,
                line=dict(color='Black', dash='dashdot'),
                xref='x',
                yref='y'
)
fig.update_layout(
    title={'text': f"Cumulative Number of Matches", 'x': 0.5,
                         'xanchor': 'center', 'font': {'size': 16}},
    xaxis_title="", yaxis_title="Cumulative Proportion", showlegend=False
)
fig.show()

In [57]:
matches_df['perc'] = matches_df['Matches']/total_matches*100
fig = px.bar(matches_df['perc'])
fig.update_layout(
    title={'text': f"Percentage of Matches at Each Rank", 'x': 0.5,
                         'xanchor': 'center', 'font': {'size': 16}},
    xaxis_title="", yaxis_title="Percentage", showlegend=False
)
fig.show()

In [58]:
comp_match_num = sum(matches_list)
custom_match_num = custom_map_df['Num Matches'].sum()
unrated_match_num = unrated_map_df['Num Matches'].sum()
spikerush_match_num = spikerush_map_df['Num Matches'].sum()
all_matches_df = pd.DataFrame([comp_match_num, custom_match_num, unrated_match_num, spikerush_match_num], 
            index=['Competitive', 'Custom', 'Unrated', 'Spike Rush'], columns=['Matches']).sort_values('Matches', ascending=False)

fig = px.bar(all_matches_df)
fig.update_layout(
    title={'text': f"Number of Matches for Each Mode", 'x': 0.5,
                         'xanchor': 'center', 'font': {'size': 16}},
    xaxis_title="", yaxis_title="Matches", showlegend=False
)
fig.show()

Agents
KD
Win Rate
Pick Rate
Average Combat Score (ACS)
First Blood Rate

In [59]:
example_agents_df = clean_data(pd.read_csv(f"https://raw.githubusercontent.com/meryembarkallah21/valorant/master/agents_data/all/agents_competitive_tier%3D3.csv"))
example_agents_df

,KD,Kills,Deaths,Assists,Win Rate,Pick Rate,ACS,First Blood,Num Matches
Agent Name,,,,,,,,,
Sage,0.78,11.5,14.7,4.2,51.9,18.8,164.0,11.7,9231.0
Raze,0.90,13.1,14.6,4.3,44.2,11.7,207.0,13.9,5763.0
Jett,1.01,14.3,14.1,4.1,53.8,10.8,210.0,15.2,5304.0
Reyna,1.01,14.7,14.6,3.9,48.5,10.3,224.0,14.7,5049.0
Viper,0.80,11.0,13.9,4.3,49.3,6.9,165.0,11.8,3417.0
Killjoy,0.86,12.6,14.7,4.3,51.6,6.6,185.0,12.2,3264.0
Phoenix,0.99,13.1,13.3,4.2,62.3,6.3,202.0,14.7,3111.0
Omen,0.85,12.6,14.7,4.5,53.4,6.0,189.0,14.0,2958.0
Sova,0.80,12.1,15.1,5.4,49.0,5.1,182.0,13.4,2499.0


In [60]:
def avg_visualization(df, name, name2 = "Agents"):
    tmp_df = pd.DataFrame(df.mean(), columns=[name]).sort_values(name, ascending=False).round(3)
    fig = px.bar(tmp_df, y=name, color=name, color_continuous_scale="Viridis")
    fig.update_layout(
        title={'text': f"Average {name} of {name2}", 'x': 0.5,
                             'xanchor': 'center', 'font': {'size': 16}},
        xaxis_title="", yaxis_title=f"{name}", showlegend=False
    )
    fig.show()

In [61]:
agents_ranks_list = ranks_list[:-1] # no Diamond 3

def visualization(df, name, name2 = "Agents", ranksList = agents_ranks_list, smoothing=True):
    if smoothing:
        df = smoother(df)
    
    c = 0
    buttons_list = []
    base_list = [False for i in range(len(ranksList))]
    for i in df.index:
        tmp_list = base_list.copy()
        tmp_list[c] = True
        buttons_list.append(dict(
            args=[{"visible": tmp_list}],
            label=i,
            method="update"
        ))
        c += 1
    
    c = 0
    fig = go.Figure()
    for i in df.index:
        tmp_df = pd.DataFrame(df.loc[i]).sort_values(i, ascending=False)
        fig.add_trace(go.Bar(x=tmp_df.index, y=tmp_df[i], name=i, 
                             visible=False if c != 0 else True,
                             marker=dict(
                                color=tmp_df[i],
                                colorbar=dict(
                                    title=f"{name}"
                                ),
                                colorscale="Viridis"
                        )))
        c += 1

    fig.update_layout(
        updatemenus=[
            dict(
                buttons=buttons_list,
                direction="down",
                pad={"r": 10, "t": 10},
                showactive=True,
                x=0.1,
                xanchor="left",
                y=1.1,
                yanchor="top"
            ),
        ]
    )

    smoothingString = "" if smoothing else "out"
    smoothingValueString = f" = {SMOOTHING}" if smoothing else ""
    fig.update_layout(
        title={'text': f"{name} of {name2} by Rank With{smoothingString} Smoothing{smoothingValueString}", 'x': 0.5,
                             'xanchor': 'center', 'font': {'size': 16}},
        yaxis_title=f"{name}", showlegend=False
    )
    fig.show()

In [62]:
def all_agents_visualization(df, name):
    avg_visualization(df, name)
    visualization(df, name, smoothing=False)
    visualization(df, name)

In [63]:
KD_list = []
winrate_list = []
agent_pickrate_list = []
acs_list = []
firstblood_list = []
for i in range(3, 3 + len(agents_ranks_list)):
    tmp_agents_df = clean_data(pd.read_csv(f"https://raw.githubusercontent.com/meryembarkallah21/valorant/master/agents_data/all/agents_competitive_tier%3D{i}.csv"))    
    KD_list.append(tmp_agents_df['KD'])
    winrate_list.append(tmp_agents_df['Win Rate'])
    agent_pickrate_list.append(tmp_agents_df['Pick Rate'])
    acs_list.append(tmp_agents_df['ACS'])
    firstblood_list.append(tmp_agents_df['First Blood'])
    
kd_df = pd.DataFrame(KD_list, index=agents_ranks_list)
winrate_df = pd.DataFrame(winrate_list, index=agents_ranks_list)
agent_pickrate_df = pd.DataFrame(agent_pickrate_list, index=agents_ranks_list)
acs_df = pd.DataFrame(acs_list, index=agents_ranks_list)
firstblood_df = pd.DataFrame(firstblood_list, index=agents_ranks_list)

KD
Kills to death ratio of agents.

In [64]:
all_agents_visualization(kd_df, "KD")

Win Rate
Which agent tends to win the most?

In [65]:
all_agents_visualization(winrate_df, "Win Rate")

Agent Pick Rate
Which agents are picked the most?

In [66]:
all_agents_visualization(agent_pickrate_df, "Agent Pick Rate")

ACS
Average combat score is probably the most representative of the impact a player has since it takes into account damage, impact of kills, and assists.

In [67]:
all_agents_visualization(acs_df, "ACS")

First Blood Rate
Which agents typically get the entry frag?

In [68]:
all_agents_visualization(firstblood_df, "First Blood Rate")

Weapons
Kills Per Match
Headshot Percentage
Average Damage Per Round

In [69]:
weapons_kills_list = []
headshot_list = []
bodyshot_list = []
legshot_list = []
weapons_damage_list = []
for i in range(3, 3 + len(ranks_list)):
    tmp_weapons_df = clean_data(pd.read_csv(f"https://raw.githubusercontent.com/meryembarkallah21/valorant/master/weapons_data/all/agents_competitive_tier%3D{i}.csv"))    
    weapons_kills_list.append(tmp_weapons_df['Kills Per Match'])
    headshot_list.append(tmp_weapons_df['Headshot'])
    bodyshot_list.append(tmp_weapons_df['Bodyshot'])
    legshot_list.append(tmp_weapons_df['Legshot'])
    weapons_damage_list.append(tmp_weapons_df['Damage Per Round'])
    
weapons_kills_df = pd.DataFrame(weapons_kills_list, index=ranks_list)
headshot_df = pd.DataFrame(headshot_list, index=ranks_list)
bodyshot_df = pd.DataFrame(bodyshot_list, index=ranks_list)
legshot_df = pd.DataFrame(legshot_list, index=ranks_list)
weapons_damage_df = pd.DataFrame(weapons_damage_list, index=ranks_list)

In [70]:
def all_weapons_visualization(df, name):
    avg_visualization(df, name, "Weapons")
    visualization(df, name, "Weapons", ranks_list, smoothing=False)
    visualization(df, name, "Weapons", ranks_list)

Kills Per Match
Average kills a weapon gets per match.

In [71]:
all_weapons_visualization(weapons_kills_df, "Kills Per Match")

Headshot Percentage
Headshot percentage given a certain weapon.

In [72]:
all_weapons_visualization(headshot_df, "Headshot")

Damage Per Round
Average damage a weapon deals for each round.

In [73]:
all_weapons_visualization(weapons_damage_df, "Damage Per Round")

Headshot Percentage with Rifles by Rank

In [74]:
fig = go.Figure()
fig.add_trace(go.Scatter(name='Vandal', x=headshot_df.index, y=headshot_df['Vandal']))
fig.add_trace(go.Scatter(name='Phantom', x=headshot_df.index, y=headshot_df['Phantom']))
fig.update_layout(
    title={'text': f"Headshot Percentage with Rifles by Rank", 'x': 0.5,
                         'xanchor': 'center', 'font': {'size': 16}},
    yaxis_title=f"Headshot Percentage"
)